In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import optuna
from scipy.stats import pearsonr

from split import prep_data_before_train, random_split, subset

phenotype = "mass"

data = pd.read_feather(f"data/processed/{phenotype}BV.feather")

df, y, ringnrs, mean_pheno = prep_data_before_train(data, phenotype)
del data
df.drop(columns = ["hatchisland"], inplace = True)
df["ringnr"] = ringnrs   

target = pd.DataFrame(y)
target["mean_pheno"] = mean_pheno
target["ringnr"] = ringnrs

folds = random_split(phenotype, num_folds=10, seed=42)

df = pd.merge(df,folds, on = "ringnr", how = "inner") 
df = pd.merge(df,target, on = "ringnr", how = "inner")

df = subset(df, num_snps=2000)

In [2]:
df.head()

,SNPa352024_T,SNPa51204_C,SNPa371349_T,SNPa86522_T,SNPa395916_T,SNPa302259_A,SNPa524601_A,SNPa483891_T,SNPa207156_T,SNPa77054_G,...,SNPa203182_G,SNPa403306_G,SNPa190407_A,SNPa212927_G,SNPa450571_C,SNPa203827_A,ringnr,fold,ID,mean_pheno
0,0,0,1,2,1,0,0,1,0,0,...,0,1,1,0,0,0,8118424,0,0.519577,32.600000
1,0,0,1,2,0,0,0,1,0,1,...,0,1,1,0,0,0,8118425,8,-1.044296,30.300000
2,0,0,0,1,0,0,0,0,0,1,...,0,0,2,0,0,0,8118426,9,0.918158,33.316667
3,0,0,0,0,0,0,0,1,0,1,...,1,0,0,0,0,2,8118429,5,-0.093239,31.300000
4,0,0,0,2,1,0,0,1,0,0,...,0,1,1,0,0,0,8118430,3,-1.010615,29.900000


In [ ]:
# Function to perform hyperparameter optimization using Optuna
def objective(trial, X_train, y_train):
    params = {
        "objective": "reg:pseudohubererror",
        "n_estimators": 600,
        "verbosity": 0, 
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 12),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 30)  
    }

    model = xgb.XGBRegressor(**params, random_state = 42) 
    
    # Perform cross-validation on the 9 inner folds
    mae_list = []
    for val_fold in range(0, 10):
        if val_fold == test_fold:
            continue
        X_train_fold = X_train[X_train['fold'] != val_fold].drop(columns=['fold'])
        y_train_fold = y_train[X_train['fold'] != val_fold]
        X_val_fold = X_train[X_train['fold'] == val_fold].drop(columns=['fold'])
        y_val_fold = y_train[X_train['fold'] == val_fold]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        mae = mean_absolute_error(y_val_fold, y_pred)
        mae_list.append(mae)
        print(f'Validation Fold {val_fold} - MAE: {mae}')
    
    return sum(mae_list) / len(mae_list)

# Outer loop: Iterate through each fold for testing
outer_mae = []
outer_corr = []
output_file = f"xgb_random_split_{phenotype}.csv"

for test_fold in range(0, 10):
    # Split data into training (9 folds) and testing (1 fold)
    X_train = df[df['fold'] != test_fold].drop(columns=['ringnr', 'ID', 'mean_pheno'])
    y_train = df[df['fold'] != test_fold][['ID', 'mean_pheno']]
    
    X_test = df[df['fold'] == test_fold].drop(columns=['ringnr', 'ID', 'mean_pheno'])
    y_test = df[df['fold'] == test_fold][['ID', 'mean_pheno']]
    
    # Perform hyperparameter optimization using Optuna
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X_train, y_train['ID']), n_trials=10, n_jobs=4)
    
    # Train the model with the best parameters on the 9 folds
    best_params = study.best_params
    best_model = xgb.XGBRegressor(
        objective="reg:pseudohubererror",
        n_estimators=600,
        verbosity=0,
        **study.best_params,
        random_state=42
    )
    best_model.fit(X_train.drop(columns=['fold']), y_train['ID'])
    
    # Evaluate on the test fold
    y_pred = best_model.predict(X_test.drop(columns=['fold']))
    test_mae = mean_absolute_error(y_test['ID'], y_pred)
    outer_mae.append(test_mae)
    corr = pearsonr(y_test['mean_pheno'], y_pred)[0]
    outer_corr.append(corr)

    print(f'Test Fold {test_fold} - MAE: {test_mae}')
    print(f'Test Fold {test_fold} - Correlation: {corr}')

    
    with open(output_file, 'a') as f:
        f.write(f'{test_fold},{test_mae},{corr}\n')

[I 2024-10-23 13:26:26,990] A new study created in memory with name: no-name-a69b0b17-246f-4c66-8333-ab19ace1d348


Validation Fold 1 - MAE: 0.7506806026625344
Validation Fold 1 - MAE: 0.7422806491840818
Validation Fold 1 - MAE: 0.7500361045936805
Validation Fold 1 - MAE: 0.7520256787134997
Validation Fold 2 - MAE: 0.6885911428604867
Validation Fold 2 - MAE: 0.6747338930040979
Validation Fold 3 - MAE: 0.7382539801587296
Validation Fold 2 - MAE: 0.6761096148480559
Validation Fold 2 - MAE: 0.6713405383275036
Validation Fold 3 - MAE: 0.7181295378613338
Validation Fold 4 - MAE: 0.7589415692890458
Validation Fold 3 - MAE: 0.716087588577699
Validation Fold 4 - MAE: 0.7726890017066684
Validation Fold 5 - MAE: 0.7850182826444922
Validation Fold 3 - MAE: 0.7056000635594711
Validation Fold 6 - MAE: 0.7836449742113688
Validation Fold 5 - MAE: 0.7656686333055722
Validation Fold 4 - MAE: 0.7720341703893593
Validation Fold 4 - MAE: 0.7594102413223602
Validation Fold 7 - MAE: 0.7500738820989467
Validation Fold 6 - MAE: 0.781817537306741
Validation Fold 5 - MAE: 0.7675737383500771
Validation Fold 8 - MAE: 0.7824667

[I 2024-10-23 13:30:57,804] Trial 1 finished with value: 0.7655400057746145 and parameters: {'learning_rate': 0.02817567287043857, 'max_depth': 5, 'subsample': 0.6677755861963288, 'colsample_bytree': 0.7892268649775355, 'min_child_weight': 28}. Best is trial 1 with value: 0.7655400057746145.


Validation Fold 9 - MAE: 0.8521889001899239
Validation Fold 6 - MAE: 0.7683992492326938
Validation Fold 8 - MAE: 0.7856870843195026
Validation Fold 6 - MAE: 0.7638462460875868
Validation Fold 1 - MAE: 0.7396736635763658
Validation Fold 7 - MAE: 0.7624585324868534


[I 2024-10-23 13:31:52,204] Trial 2 finished with value: 0.7598450027453633 and parameters: {'learning_rate': 0.013488855185003186, 'max_depth': 7, 'subsample': 0.7494111847538755, 'colsample_bytree': 0.7921994720260556, 'min_child_weight': 1}. Best is trial 2 with value: 0.7598450027453633.


...

Validation Fold 9 - MAE: 0.8614753199920132
Validation Fold 8 - MAE: 0.7864543210740323
Validation Fold 8 - MAE: 0.796789234131802
Validation Fold 7 - MAE: 0.7550225207877371
Validation Fold 0 - MAE: 0.7400399650818628


[I 2024-10-23 13:52:42,386] Trial 5 finished with value: 0.7569940767533233 and parameters: {'learning_rate': 0.01767145610286903, 'max_depth': 9, 'subsample': 0.706349035808308, 'colsample_bytree': 0.6949893360846221, 'min_child_weight': 19}. Best is trial 4 with value: 0.7512596476190169.


Validation Fold 9 - MAE: 0.8501609290978075


[I 2024-10-23 13:52:43,731] Trial 6 finished with value: 0.7632389243855209 and parameters: {'learning_rate': 0.0011333354735192968, 'max_depth': 11, 'subsample': 0.7640651147261996, 'colsample_bytree': 0.9748188319079087, 'min_child_weight': 27}. Best is trial 4 with value: 0.7512596476190169.


Validation Fold 9 - MAE: 0.885012499021354
Validation Fold 8 - MAE: 0.7895137780909508
Validation Fold 2 - MAE: 0.6918384011180084
